# ICC305 Avaliação de Desempenho - 2023/1

## Atividade 3

**Giovanna** Andrade Santos - 22052563

**Marcos** Avner Pimenta de Lima - 21852448

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import math

from scipy.stats import chisquare
from scipy.stats import chi2_contingency

from ad_library import myfunctions
from collections import defaultdict

### Gerador G5 (Python)

In [5]:
import random

class G5RandomGenerator():
    
    def __init__(self, seed=0):
        self.__n0 = seed
        random.seed(self.__n0)
    
    # gera o próximo número com base no último número gerado
    def next(self):
        return random.random()
    
    # reinicia o gerador
    def reset(self):
        random.seed(self.__n0)

### Geração de uma V.A. Exponencial

In [72]:
def va_exp(gerador, beta):
    U = gerador.next()
    return -beta * math.log(1 - U)

In [48]:
#nº de clientes
n1 = 10 ** 3
n2 = 10 ** 6
n3 = 10 ** 9
n4 = 10 ** 12

#taxa de entrada --> 9 clientes/s
tc = 9

#taxa de serviço --> 10 clientes/s
ts = 10

# https://introcs.cs.princeton.edu/python/43stack/mm1queue.py

In [62]:
def va_exp(beta, gerador):
    return -beta * math.log(1 - gerador.next())

# Accept float command-line arguments lamb and mu. Simulate an
# M/M/1 queue with arrival rate lamb and service rate mu.
def mm1(n=1000, tc=9., ts=10., gerador=G5RandomGenerator()):
    lamb = 1. / tc
    mu = 1. / ts
    fila = list() # fila de espera
    espera = 0
    
    prox_chegada = va_exp(lamb, gerador) # computa o tempo da proxima chegada.
    prox_atend = prox_chegada + va_exp(mu, gerador) # computa o tempo da conclusao do proximo atendimento.
    
    # Simulate the M/M/1 queue.
    while n > 0:

        while prox_chegada < prox_atend:
            # simula a proxima chegada
            fila.append(prox_chegada)
            prox_chegada += va_exp(lamb, gerador)

        # Next event is a service completion.
        if fila:
            chegada = fila.pop(0)
            espera += prox_atend - chegada

        # atualiza a fila de espera.
        if fila:
            # se a fila não estiver vazia, o termino do proximo atendimento 
            # é incrementando com o ts
            prox_atend += va_exp(mu, gerador)
        else:
            # se a fila estiver vazia, o termino do próximo atendimento
            # é incrementando com o (tc + ts)
            prox_atend = prox_chegada + va_exp(mu, gerador)
        
        n -= 1
    return espera

In [64]:
g5 = G5RandomGenerator()
g5.reset()

for n in [n1, n2, n3]:    
    tot_espera = mm1(n, tc, ts, g5)
    print(f'N: {n}')
    print(f'Espera total: {tot_espera}')
    print(f'Espera Média: {tot_espera / n}')
    print()

N: 1000
Espera total: 674.3278830878813
Espera Média: 0.6743278830878814

N: 1000000
Espera total: 1003381.2442020037
Espera Média: 1.0033812442020038

N: 1000000000
Espera total: 1001133126.5219395
Espera Média: 1.0011331265219394

